
<span type="title">访问控制权限、类的重用</span> | <span type="update">2018-07-18</span> - Version <span type="version">2.4</span>
    
    
<span type="intro"><p class="card-text">本章第一部分讲解Java的权限控制问题，首先讲解了文件、构件、群组和包的结构，接着从方法和类的角度讲解了几种不同权限控制关键字，以及其作用域和使用方法。其中提到了默认包的问题、变量配置的问题等细节。</p><p class="card-text">在第二部分讲解了类重用的方法，其中包含组合、继承、结合、代理等不同方法，这些方法适用于不同的模式。对于组合，介绍了变量初始化时机和权限字的使用问题。对于继承，介绍了有无参数的初始化构造、protected关键字在继承中的作用、继承重载方法和@Override名称屏蔽。之后，介绍了组合使用继承和结合的技术，在什么情况下使用继承或者结合，提到了向上转型。</p><p class="card-text">第三部分，介绍了final关键字在数据、方法参数、方法、类上的不同效果和作用。final关键字也是权限关键字的一种，对于参数进行限制较为常用。在最后，本章梳理了有继承的情况下，一个类被调用时的代码执行顺序问题。权限和重用有密切的关系，比如向上转型、final方法和类、protected关键字，这些权限设置都是用来限定继承的，虽然继承在真实情况下用的并不太多（更多的是组合）。</p></span>

# 对象的权限控制问题

在面向对象语言中，一个较大的问题是类方法的权限问题，如果对一个对象而言，其所有的域和方法都面向使用者开放，这势必造成混乱，更为麻烦的事，对于类的维护者而言，其几乎无法重构一个已经发布的类，因为类的任何域或者方法都有可能被其余程序调用。Python选择了一种“大家都是成年人”的方式，完全放弃了权限控制，这很糟糕。而在高级的Python编程专著中，往往会有一部分章节用来介绍如何将权限控制加回去的。

Java面临的第一个问题是变与不变的问题，对于用户的接口，也就是API，不变，使用 `public` 关键字声明方法或者类，对于内部的接口，使用 `private protected` 来进行其余的分配。这样就可以在不影响接口的情况下大刀阔斧的修改类，进行重构了。

Java面临的第二个问题是，如何区分Array这个类，如果系统实现了它，而用户又实现了一遍？对于Python而言，其默认用户声明的具有较高的优先级（本地目录），然后查找系统自带的库。对于C而言，使用 `include <stdio.h> 或者 include "stdio.h"` 来进行类似的优先级判定。对于Java而言，则是使用 `package 和 import` 进行管理。

# 文件、构件、群组和包

`.java` 文件包含Java源代码，称之为一个编译单元。

对于每一个 `.java` 的文件，当使用 `javac` 工具时，会在同目录生成一个 `.class` 的 JVM 二进制字节码，这两个后缀为 `.java, .class` 的文件合起来称之为一个类的构件。

对于一个构件而言，更改只需要发生在 `.java` 文件即可，当系统调用时，会自动进行 `.class` 文件的更新和处理，不用手动进行。

对于 `.java` 源文件而言，其一般包含一个 `public class ClassName` 的类，这个文件的名称也必须为 `ClassName.java`。并且在这个文件中，不能出现两个 `public class ` 类，虽然可以出现多个 `class` 类，其中这些其余的类具有包访问权限，供主要的类进行调用和处理，在包外不可调用。

当把多个 `.java .class` 类构件放在一个文件夹下，就称之为一个群组。

**包：库单元**

如何告诉Java编译器这些Class文件是一个包呢？对于Python而言，一般需要创建一个空文件夹声明这是一个包，对于Java而言，则必须对每个 `.java` 源文件的**首个非注释行**声明 `package com.mazhangjing.util` 这样一句话，告诉Java这个构件属于 `com.mazhangjing.util` 这个包。也可选择将这个群组打包压缩成 `.jar` 文件，然后将这个文件添加到 `CLASSPATH` 变量中使用。

对于 `com.mzhangjing.util` 包而言，其不是说存在这样一个文件夹叫做 `com.mazhangjing.util`，Java会将句号转换为斜杠，因此这其实是一个三级文件夹，所有的构件都存放在 `com/mazhangjing/util` 这个文件夹下。此外，为了让Java找到这个包，在 `CLASSPATH` 变量中添加 `com` 文件夹存在的父文件夹。

**包和编译的注意事项**

注意，不像Python之类语言会从当前目录开始查找包并自动导入，在 `CLASSPATH` 中如果不存在 `.` 这个目录的话，JAVA是会默认忽略本地文件的。也就是或，当添加 `.` 到变量时，你可以在 `util` 这个目录下执行程序，程序会在 `import util.*` 的时候寻找本地文件夹下的类。

此外，需要注意，对于 `javac` 而言，如果对一个包中的类进行执行的话，如果找不到类，那么添加其包名可能有奇效。`javac com.mazhangjing.util.Print.java` 这样，而不是 `javac Print.java`。因为后者包含 `package com.mazhangjing.util`，在进行编译的时候，它是无法找到当前文件夹下的`/com/mazhangjing/util/Print`这个目录并将自己放进去的。

**导入包中的类**

使用 `import com.mazhangjing.util.Print` 可以导入 `Print` 类，被导入的类的 `com` 文件夹必须在 `CLASSPATH` 能够找到的位置。而使用的这个文件，则可以位于任何地方。

使用 `import com.mazhangjing.util.*` 可以导入 `util` 目录下的所有类，在使用的过程中，需要指明类。

使用 `import static com.mazhangjing.util.Print.*` 可以导入 `Print.class` 类中的所有静态方法，注意，不能用这种写法导入其余方法，一般而言，不能对一个类加`.*`。

```java
/com/mazhangjing/Print.java

package com.mazhangjing;
import java.io.*;
/**
 * Print shortcuts for Study and play Java!
 * @author Corkine Ma
 * @author www.mazhangjing.com
 * @version 0.1
*/
public class Print{
	/**
	 * use for print something by calling System.out.println()
	 * @param obj Object-something
	*/
	public static void print(Object obj){
		System.out.println(obj);
	}
	
	public static void print(){
		System.out.println();
	}
	public static void printnb(Object obj){
		System.out.print(obj);
	}
	public static PrintStream 
	printf(String format, Object... args){
		return System.out.printf(format, args);
	}
	public static void main(String[] args){
		print("Hello");
	}
	
}

import com.mazhangjing.Print;
Print.print("hello");

import com.mazhangjing.*;
Print.print("hello");

import static com.mazhangjing.Print.*
print("hello");
```

# 权限关键字

如下代码所示：

```java
/com/mazhangjing/Power.java
package com.mazhangjing;
import static com.mazhangjing.Print.*;
//import static Print.*; can't work
public class Power{
	public void pubA(){
		print("Hello from Power class pubA");
	}
	private void priA(){
		print("Hello from Power class priA");
	}
	protected void proA(){
		print("Hello from Power class proA");
	}
}

/somewhere/somename.java
import com.mazhangjing.Power;
class LittlePower extends Power {void proB(){ super.proA();}}
Power a = new Power();
a.pubA();
//a.proA(); //错误
//a.priA(); //错误
LittlePower b = new LittlePower();
b.pubA();
b.proB();
//b.proA(); //错误
```


## 方法的权限关键字

权限一共有四种，从小到大依次是：

- private 权限私有，仅本文件可以访问
- 包访问权限 权限仅限本包内的成员访问
- protected 可以跨越包在别处使用，不过限制只能在继承的子类内部使用，不能直接调用
- public 开放权限，任意位置均可调用

注意上面这个包中的Power类以及其在某个文件中的扩展子类和调用。

很有意思，对于 `public` 而言，其可以在任意地方调用（前提是导入了相关的类，如果不是 static 的话还需要构造这个类对象，下同）。

对于 `private` 而言，其在除了这个类文件的任何地方都不可调用，甚至是包里的其他类也不可以调用。也就是说，其权限锁定在其文件内。

如果不写任何关键字，那么默认这个方法的权限在整个包内，也就是说，属于 `com.mazhangjing` 这个包的所有类文件都可以调用这个方法（再啰嗦一遍，需要import，并且在包内也必须写清楚包名，不能写`import Print`而应该是`import com.mazhangjing.Print`）。

对于`protected`而言，需要说明的是，这是一种专门用来解决跨包，但是又继承了父类的情况，如`LittlePower`而言，被保护的方法可以在其类内部使用，它可以将其包装起来作为自己的公共方法，但是不能使用`proA`,因为这样就变成了继承和重载，不能对于保护方法进行重载，不过换个名字还可以用，所以说，保护方法的权限比公共方法小，比包方法大。

**默认包问题**

如果不声明 `package xxx.xxx.xxx` 的话，Java将你运行的类看作是默认包，这个包可能存在的问题是，你可以对于任意使用 `void methodName(){}` 的方法，也就是具有包权限的方法进行访问，这可能不是你希望的。

```java
/xxx/Cake.java
class Cake {
	public static void main(String[] args){
		Pie x = new Pie();
		x.f();//Pie.f()
    }}

/xxx/Pie.java
class Pie {
	void f() {System.out.println("In Pie.f()");}
}
```

## 类的权限关键字

 类只有两种访问权限，`public` 和 包访问权限。如果不希望包的用户访问此类，可以使用 包访问权限，反之，使用公共权限。
 
 ```java
/com/mazhangjing/Copyright.java
package com.mazhangjing;
import static com.mazhangjing.Print.*;
class Copyright{
	public static void getCopyright(){
		print("(c) Marvin Studio 2018");
	}
}
/com/mazhangjing/Power.java
public class Power{
	public Power(){
		print("Init Power...");
	}
	public static void main(String[] args){
		print("Hello from Power class!");
		Copyright.getCopyright(); //可以，在同一包中，具有相同权限
	}
}
/somewhere/somefile.java
Copyright.getCopyright(); // 错误，不可在外部访问
```

# Java类的重用

## 类的组合

组合指的是将多个类组合在一起使用，新的类一般包含着其余类的实例，这样就可以在这个类中调用那些类的方法了，这种方法其使用了类的功能，复用了代码而非形式。

```java
import static com.mazhangjing.Print.*;
public class Bird {
	public static void main(String[] args){
		getInfo();
	}
	public static void getInfo(){
		print("Hello from Bird.class");
		Eyes eye = new Eyes();
		print("The bird have a " + eye);
		Foot foot = new Foot();
		print("The bird have " + foot.numOfIt() + " foots.");
		print("The bird's foot is " + foot.colorOfIt() + ".");
	}
}
class Eyes {
	private String info = "blue eyes";
	public String toString() {return info;}
}
class Foot {
	private int num; 
	private String color;
	public int numOfIt(){return num;}
	public String colorOfIt(){
		if (color == null) {color = "red";}
		return color;
	}	
}
```
```
Hello from Bird.class
The bird have a blue eyes
The bird have 0 foots.
The bird's foot is red.
```
**变量初始化时机**

如上所示，Bird的`getInfo()`方法组合了Eyes和Foot这两个类，在其内部创建了实例（静态的话可以不用）并且调用了其方法。这是组合的实例。需要注意，对于初始化的问题，一般而言有无个地方可以用于初始化：

- 类的域，当定义时即赋值
- 域上下文，同上
- 构造器构造期间
- 当使用的时候（惰性调用，占用资源少，比如Foot的colorOfIt()调用）
- 使用实例进行初始化

**组合的public和private问题**

说回正题，对于组合而言，一般我们对实例的对象使用 `private` 来屏蔽外界访问，但是，在某些情况下，使用 `public` 可以让抽象更加的清晰。比如下面这个例子，从 Door 中引出 Window: `car.left.window.rollup()` 非常的自然，因此使用 `private` 较好。

```java
//show what we should use in a combination, public or private
import static com.mazhangjing.Print.*;
public class Car {
	public Engine engine = new Engine();
	public Wheel[] wheel = new Wheel[4];
	public Door left = new Door(), right = new Door();
	public Car() {
		for (int i = 0; i < 4; i++){
			wheel[i] = new Wheel();
		}
	}
	public static void main(String[] args){
		Car car = new Car();
		car.left.window.rollup();
		car.wheel[0].inflate(72);
		car.engine.service(true);
	}
}
class Engine {
	public void start(){}
	public void stop(){}
	public void service(boolean t) {
		print("Engine service on..." + t);
	}
}
class Wheel {
	public void inflate(int psi){}
}
class Window {
	public void rollup(){}
	public void rolldown() {}
}
class Door {
	public Window window = new Window();
	public void open() {}
	public void close() {}
}
Engine service on...true
```

## 类的继承

继承是一种对于类形式和功能全方位的重用。在Java中，任何时候都在继承，当你没有指定被继承对象时，其默认为`Objcet`类。继承使用 `class Son extends Father` 来声明被继承的类。

继承可以在子类中对父类的方法进行重载，这样调用的话，使用的就是子类的方法。而那些没有在子类进行重载的方法，默认会调用父类的方法，这样就实现了重用。继承有几个重要问题，比如初始化和其顺序问题。

**没有参数的继承初始化及其顺序**

```java
import static com.mazhangjing.Print.*;
public class A {
	public static void main(String[] args){
		D dins = new D();
	}
}
class B {
	protected String info = "In B";
	B(){print("Init B");}
}
class C {
	C(){print("Init C");}
}
class D extends B{
	C cins = new C();
	D(){print("Init D");}
}

D dins = new D();
Init B
Init C
Init D
```


如上所示，对于没有参数的继承，首先调用父类进行了初始化，如果父类也有父类，那么找到最高处的类依次调用其构造器进行初始化。继承的父构造器初始化的优先级最高，比域和自身构造器都高。

**有参数的继承初始化**

如果有参数，则需要手动指定。比如：

```java
import static com.mazhangjing.Print.*;
public class A {
	public static void main(String[] args){
		Chess c = new Chess();
	}
}
class Game {
	Game(){print("Game init");}
}
class BoardGame extends Game {
	BoardGame(int i){print("BoardGame init with num " + i);}
	BoardGame(){print("BoardGame init");}
}
class Chess extends BoardGame {
	Chess(){
		//super(11);
		print("Chess init");
	}
}

Chess x = new Chess();

Game init
BoardGame init with num 11
Chess init
```

如上所示，Chess的初始化需要BoardGame，BoardGame必须有一个参数，因此如果不调用`super(11)`的话，Java不会默认调用父构造器，会出错。同理，因为BoardGame的父构造器不需要参数，因此其不用写`super`，Java会自动调用，并且按照正确顺序返回。

总之，对于任何被继承的类，在初始化的时候都需要关注父类的构造器是否需要接受参数，如果需要，则必须手动调用 `super` 方法来构造父类。有一点需要注意，如果可选父类构造器，而没有手动传递 `super` 则会通过没有参数的构造器构造。这里可以看出手动构造的必要之处，我们必须选择父类被构造的状态。

**继承和protected关键字的使用**

对于继承而言，`protected` 能够保护在类之外的地方，继承的子类可以在其内部调用父类的方法和域。如下的`set`方法，只对于继承的子类可用。

```java
//use to show protected in inherit
import static com.mazhangjing.Print.*;
public class Orc extends Villain {
	private int orcNumber;
	public Orc(String name, int orcNumber){
		super(name);
		this.orcNumber = orcNumber;
	}
	public void change(String name, int orcNumber){
		set(name);
		this.orcNumber = orcNumber;
	}
	public String toString(){
		return "ORCNUMBER: " + orcNumber + ", " + super.toString();
	}
	public static void main(String[] args){
		Orc x = new Orc("Cokrine",20122212);
		print(x);
	}
}
class Villain {
	private String name;
	public Villain(String name){
		this.name = name;
	}
	public String toString(){
		return "I'm a Villain and my name is " + name;
	}
	protected void set(String name){this.name = name;}
}
```

**继承中的重载：名称屏蔽**

Java允许强制运算符重载，如下是一个例子，展示了使用`@Override`来重载时发生的错误。

```java
//show how to use @Override and avoid to havn't override the super class.
import static com.mazhangjing.Print.*;
public class Fruit {
	public static void main(String[] args){
		print("Fruit...");
		Apple a = new Apple();
	}
	String nameOfIt(){
		return "Fruit...";
	}
	String nameOfIt(int n){
		return "Fruit..." + n;
	}
}
class Apple extends Fruit{
	@Override 
	String nameOfIt(String s){
		return "Apple" + s;
	}
}
```

对于C++而言，当一个类从另一个类中进行继承，那么被继承的类中的方法的不同版本是被屏蔽的（相同方法名称不同参数），而对于Java，如果基类有一个 `doA(int,float) 和 doA(int)` 两个方法，如果子类重载了 `doA(int,float)` 的话，那么另一个 `doA(int)` 亦然可以在子类中被访问。

使用 `@Override` 可以让你清楚的强制去重载某个类，告诉Java你需要重载父类中的这个同名方法，如果父类不存在这个方法，那么就会报错。这样提高了程序的清晰程度，方便看到哪些类是重载的。

注意，继承重载了域和所有的父类方法。

## 使用代理

代理是一种将继承和组合的中庸选择，注意，其和`组合和继承的结合`不同，其本身是一种妥协。对于很多情况，我们不能直接使用抽象，比如控制器和电脑，电脑明显包含控制器，但是也有别的东西。这种方式可以将多个类和继承类创建实例放置在一个父组合框架中，就像`组合和继承的结合`那样，不过有时候，通过代理解决更加简单。

```java
import static com.mazhangjing.Print.*;
public class Spaceship{
	Control ctr = new Control();
	Spaceship(){
		print("Init Spaceship");
	}
	void left(int i){
		ctr.left(i);
		print("Go left by Spaceship");
	}
	void right(int i){
		ctr.right(i);
		print("Go right by Spaceship");
	}
	public static void main(String[] args){
		Spaceship x = new Spaceship();
		x.left(27);
	}
}
class Control {
	void left(int i){
		print("Go left, controler!");
	}
	void right(int i){
		print("Go right, controler!");
	}
}

Init Spaceship
Go left, controler!
Go left by Spaceship
```

这个例子非常好玩，其中Spaceship是Control的代理，其在自己内部实例化了一个控制器对象，并且操纵了这个对象所有方法并重新实现。和组合和继承的结合不同，如果我们进行组合的类和这些实例化的类之间关系不密切，比如家和锤头，家没必要实例化锤头所有方法，而飞船和控制器关系很密切，就像父子，因此代理更加合适。

## 组合和继承的结合

如下代码：

```java
import static com.mazhangjing.Print.*;
public class PlaceSetting extends Custom {
	private Spoon sp;
	private Knife kn;
	private DinnerPlate pl;
	public PlaceSetting(int i){
		super(i+1);
		sp = new Spoon(i+2);
		kn = new Knife(i+3);
		pl = new DinnerPlate(i+4);
		print("PlaceSetting constructor");
	}
	void dispose(){
		print("Disposing Custom-Placesetting");
		sp.dispose();
		kn.dispose();
		pl.dispose();
		super.dispose();
	}
	public static void main(String[] args){
		PlaceSetting x = new PlaceSetting(9);
		try{ /*something*/}
		finally {x.dispose();}
	}
}
class Custom {
	Custom(int i){print("Custom have " + i + " people.");}
	void dispose(){
		print("Disposing Custom");
	}
}
class Plate {
	Plate(int i){print("Plate constructor with " + i);}
	void dispose(){
		print("Disposing Plate");
	}
}
class DinnerPlate extends Plate {
	DinnerPlate(int i){
		super(i);
		print("DinnerPlate constructor with " + i);
	}
	void dispose(){
		print("Disposing Plate");
		super.dispose();
	}
}
class Utensil {
	Utensil(int i){
		print("Utensil constructor with " + i);
	}
	void dispose(){
		print("Disposing Utensil");
	}
}
class Spoon extends Utensil {
	Spoon(int i){
		super(i);
		print("Spoon constructor with " + i);
	}
	void dispose(){
		print("Disposing Utensil-Spoon");
		super.dispose();
	}
}
class Knife extends Utensil {
	Knife(int i){
		super(i);
		print("Knife constructor with " + i);
	}
	void dispose(){
		print("Disposing Utensil-Knife");
		super.dispose();
	}
}


Custom have 10 people.
Utensil constructor with 11
Spoon constructor with 11
Utensil constructor with 12
Knife constructor with 12
Plate constructor with 13
DinnerPlate constructor with 13
PlaceSetting constructor
Disposing Custom-Placesetting
Disposing Utensil-Spoon
Disposing Utensil
Disposing Utensil-Knife
Disposing Utensil
Disposing Plate
Disposing Plate
Disposing Custom
```

其使用了继承和组合的方式来构建一个对象。对于很多真实情况，这样做是符合常理的。这种技术可以用来构建极其复杂的类，而不用拘泥于要创建一个完美抽象的对象或者使用一堆基础工具来进行拼凑。我们可以清晰的看到继承的层次，它就是按照“类”这一概念，就好比超市分门别类的物品一样进行放置的。而我们要搭建小屋，自然要买一堆东西回来，这就像是组合。

这种方式赋予了面向对象方法巨大的灵活性。

**确保正确清理**

一般而言，Java的垃圾回收器会自动对于没有引用的对象进行清理，但是，在某些情况下，使用手动清理还是有好处的。因为一旦交给垃圾处理器，它做什么就很难知道了。

对于上述使用了继承和组合的模式，清理需要对本类创建的所有对象进行清理，调用各自的清理方法，对于有继承的类，其清理方法应该包括对于父类的清理，这有助于防止某个子对象依赖另一个子对象。如上的disponse()方法就是一个很好的例子，注意：一般将其放置在 `try...catch...finally` 的末尾处。

## 继承和组合的选择

打个比方，继承就好像一心一意，优点是可以全面控制对象，可以为它扩充方法，缺点是只有一个对象。代理就好像婚内出轨，你不用看对象脸色，创建一个实例后可以选择性的代理其方法，也可以搞别的对象过来玩，但是核心还是这个主要对象。优点是提供了一个更广阔的编程环境，可以从更加宏观层面对对象们进行操作，权利更大，更自由，出了事可以自己扛着。组合就好像花花公子，对象很多，对每个对象了解都不深入，主要用来做一个容器，协同管理，出了事还要找对象自己解决，权利不大。

总的来看，继承和代理更像，主要关注各自的对象，组合将其拢在一起，主要关心整体的结构。

从另一个角度而言，不存在代理这种模式。继承就是套用它类结构，组合就是实例化它类。它们的区别在于，继承是顺序结构（纵向），而组合是平面结构（横向）。继承能够让组合中的类结构和层次更加清晰。

所谓的代理，就是阉割的组合，其含有的对象只有一两个，重点在于手动实例化了这些对象的方法，模仿了继承的结构，提高了深度。

如果一个类含有很多类实例，这些类很像，并且需要对其循环计数，那么代理是个不错的主意，手动实例化这些实例的方法，同时提供了在这些实例之外进行循环计数的其他结构，比如说一个计数器类。当然，也可以使用继承，然后在其中手动实例化一个计数器类，不过，将这些类的容器看作其子孙，未免在逻辑上难以理解。对于这种使用组合太大的问题，使用代理就恰到好处。

——————————————————————————

以上是题外话。说回继承和组合，继承代表了`is-a`的一一对应关系，对于上上个例子，交通工具包含汽车，但是汽车是一个子集，所以不符合这种对应，使用继承毫无意义，你几乎无法想到汽车需要继承交通工具的方法的意义何在。

组合代表了`have-a`的关系。被组合的实例被包含在一个更大的抽象概念中。

因此，使用这个区别来决定使用组合还是继承就很好。但是，因为继承在OOP中使用的实在是很少，所以，我们说，除了一定需要`向上转型`的情况，尽量不要使用继承。

**向上转型**

什么是向上转型？简要而言，就是被继承的子类访问父类的方法，不是通过自身的重载和调用，要做到这一点很简单，在父类提供一个静态方法，参数为其自身的类，在这个方法中调用自身的其他方法和域，这样，子类就可以使用这个静态方法来调用父类方法，比如：

```java
//up transformation
import static com.mazhangjing.Print.*;
public class Frog extends Amphibian {
	public static void main(String[] args){
		Frog x = new Frog();
		Amphibian.getColor(x);
		x.getColor(x);
	}
	//@Override
	static void getColor(Amphibian a){
		print("In Frog");
		print(color +", "+ ext);
	}
 }
 class Amphibian {
	static String color = "red";
	static private String info = "can access private";
	static protected String ext = "can access protected";
	static void getColor(Amphibian a){
		print("In Amphibian");
		print(color+", "+info+", "+ext);
	}
 }
In Amphibian
red, can access private, can access protected
In Frog
red, can access protected
```

这里的getColor本来只接受Amphibian类型对象，而现在接受了Frog对象，因此称之为向上转型。

# final 关键字

final通常指的是无法改变的这一含义。对于数据、方法、类都可以使用这一关键字，类似于C中的const。之所以把它放在这里，是因为final作用于方法和类的时候，更像是对于继承的限制，而final作用于数据和参数的时候，更像是表示一种对于数据的限制。总而言之，final代表着一种对于Java组件使用的权限，不论是数据、参数还是继承的方法和类。

## final 数据

下列代码展示了 final 在数据使用上的问题。

```java
//show the difference of final and static final
import static com.mazhangjing.Print.*;
import java.util.*;
public class Num {
	//static final int O; variable O not initialized in the default constructor
	static final int A = 233;
	static int B = 234;
	static Integer C = 235;
	static final Integer D = 235;
	
	public static void main(String[] args){
		print("From Num.class");
		print(A+","+B+", "+C+", "+D);
		//A = A + 1;  cannot assign a value to final variable a
		B = B + 1;
		C = C + 1;
		//D = D = 1; cannot assign a value to final variable D
		print(A+","+B+", "+C+", "+D);
		
		Random rand = new Random();
		final int E = rand.nextInt(10);
		print(E);
		//E = 233; cannot assign a value to final variable E
	}
	
}
```

简而言之，其一，必须在定义处或者构造器内进行初始化，否则不可使用，比如 第一句注释。

其二，其更多的作用于基本类型而不是包装器，因为包装器定义了一个引用，引用不变不意味着引用的对象不变，这类似于C中const指针，指针不可重新赋值，但是其指向的地址的内容可以改变。

如上的 E 变量，其每次都是随机生成的，也就是这个意思，final 的唯一作用就是禁止赋值，而无法阻止其值改变。

其三，常用的写法是 `public final static int xxx` 这样声明了这个类型的数字变量是静态唯一、不变、公开访问的，因此称之为常量，一般使用大写。

## 空白 final

final可以在定义的时候进行赋值，也可以在构造器内赋值，后者给与了其极大的灵活性，同时不失其本身不变的意味。

```java
//use to show black final's power
import static com.mazhangjing.Print.*;
public class BlankFinal {
	private final String name;
	private final int age;
	public static void main(String[] args){
		print("Hello in BF.class");
		new BlankFinal();
		BlankFinal bf = new BlankFinal("Corkine",22);
		print(bf.name);
		//bf.name = "Marvin"; cannot assign a value to final variable name
	}
	public BlankFinal(){
		this.name = "Nobody";
		this.age = 0;
	}
	public BlankFinal(String name, int age){
		this.name = name;
		this.age = age;
		print("Initing with info "+name + ", " + age);
	}
}
Hello in BF.class
Initing with info Corkine, 22
Corkine
```

如上代码展示了name和age，这两个变量当初始化构造器后不可更改，但是在构造器初始化时，可以根据需要的对象来设置，这样很方便。

## final 参数

final可以用作方法的参数，和C不同，C的函数参数传递的都是值，而Java的方法传递的都是对象（除了基本类型）。因此，能够禁止在方法中修改是一个很重要的功能。

```java
//use to show final parameter
public class Final {
	void with(final Apple a){
		//a = new Apple(); 
        // final parameter a may not be assigned
	}
	void without(Apple a){
		a = new Apple();
	}
	public static void main(String[] args){
		Apple a = new Apple();
		Final x = new Final();
		x.with(a);
		x.without(a);
	}
}
class Apple {}
```

## final 方法

final 用作方法暗示了这个方法的最终性。final 和 private 关系很密切，起始 private 默认就相当于 private final，其不可被别的类调用，无法覆盖它。

```java
//show how final works with method
import static com.mazhangjing.Print.*;
public class Overriding extends WithFinals {
	//@Overriding //method does not override or implement a method from a supertype
	private final void f() {
		print("Overriding final private f()");
	}
	//@Override g() in Overriding cannot override g() in WithFinals
	//overridden method is final
	/*
	public final void g() {
		print("Overriding private g()");
	}*/
	@Override
	public final void p() {
		print("Overriding private p()");
	}
	public static void main(String[] args){
		Overriding x = new Overriding();
		x.f();
		x.g();
	}
}
class WithFinals {
	private final void f() {
		print("WithFinals with final private f()");
	}
	public final void g() {
		print("WithFinals with final public g()");
	}
	public void p() {
		print("WithFinals with public p()");
	}
}
```

如上代码，对于继承而言，声明 final 意味着你不想让别人去继承这个方法来做别的事情。

你可以在一个 public 的父方法中重载，并且添加 Override 标识符和 final 关键字，说明这个继承到你这儿结束，不允许进一步的继承。

但是对于 public final 的父类，子类不允许继承。如 g() 所示的那样。

对于私有方法而言，因为其默认就不允许继承，所以对于一个添加了 private final的父类，子类也可以写一个同名的 private final，这不是方法重载。

总结而言，当没有 final 声明方法时，可以重载。当有 final 声明方法时，对于 public，不可重载，对于 private 而言，可以使用，但没有发生重载，系统不会提醒出错。

在很多情况下，不希望别人继承，指定 final 很有用。

## final 类

final 类的目的很明显，不可继承，独此一份。

```java
//final class
public class NoFather extends Father{ //error: cannot inherit from final Father
	
}
final class Father {	}
```

注意，final 方法和 final 类更多的是对继承而言的， 其都在控制继承是否能够发生，如果类整个不希望被继承，那么使用 final 类，如果类中的某些方法不希望被重载，则使用 final 方法。

# 含有继承的类的初始化顺序

在之前介绍过静态域、构造器的初始化顺序，现在添加继承，看看结果怎么样。

```java
//show the order of var init..
import static com.mazhangjing.Print.*;
public class Bee extends Animal {
	static int q = 0;
	Bee(){
		print("static q is init before, it is "+q);
		print("I'm in the Bee.class constructor");
	}
	public static void main(String[] args){
		print("I'm runing the first line code in Bee.main");
		Bee x = new Bee();
		
	}
}
class Animal {
	int i = 1;
	static int p = 2;
	Animal(){
		print("static int p is init before now, it is"+p);
		print("int p is init before now, it is"+i);
		print("I'm in Animal.class constructor");
	}
	
}
I'm runing the first line code in Bee.main
static int p is init before now, it is2
int p is init before now, it is1
I'm in Animal.class constructor
static q is init before, it is 0
I'm in the Bee.class constructor
```

可以看到，和C非常不同，所有的类，只有当其被调用时才被加载，这发生在两种情况下，其一，其中的静态方法被调用，其二，一个此类的对象被构造。

在调用后，首先发生的肯定是类的静态域定义，之后是动态域定义，之后是？因为现在添加了父类，所以本来应该是构造器，因为有父类，并且这里隐式声明了super()，所以调用父类，当调用父类时，我们看到，父类的动态域、静态域都被调用了，之后是父类的构造器，之后才回到子类的构造器。

以上就是添加了继承后的类在被调用时的顺序。

**main方法用作调试的细节**

main方法是用作类进行调试的，因为其是一个 static 方法，所以不能使用类中的任何非static对象和域，其正确使用方法是通过类的构造器创建一个类的对象，然后对这个对象进行某些行为来测试其表现。就像在这些例子中我们的使用方式一样。